In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR


In [8]:
#read the model
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(1, 4), stride=(1, 2))
        self.bn1 = nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 4), stride=(1, 4))
        self.dropout1 = nn.Dropout(p=0.25)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1, 2), stride=(1, 2))
        self.bn2 = nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.pool2 = nn.MaxPool2d(kernel_size=(1, 4), stride=(1, 4))
        self.dropout2 = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(47104, 128)
        self.dropout3 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 6)


    def forward(self, x):
        x = torch.unsqueeze(x, 1)
        #print('x:', x.shape)
        x = self.conv1(x)
        #print('conv1:', x.shape)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        #print('conv2:', x.shape)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        #print('flatten:', x.shape)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

model = EEGNet()

In [9]:
import numpy as np

def sliding_window(signal, window_size, step_size):
    n_channels, n_samples = signal.shape
    n_windows = int((n_samples - window_size) / step_size) + 1
    windows = np.zeros((n_channels, window_size, n_windows))
    for i in range(n_windows):
        windows[:, :,i ] = signal.iloc[:, i*step_size:i*step_size+window_size]
    return windows

signal = pd.read_csv("C:\\Users\\a1882\Desktop\EEG\eegdata\\raw\lefthand_zyy_05_epocflex_2023.03.22t16.50.54+08.00.md.bp.csv")
signal = pd.DataFrame(signal)
# 定义滑窗大小和滑动步长
window_size = 3000
step_size = 100

# 对信号进行滑窗处理
windows = sliding_window(signal, window_size, step_size)

# 输出滑窗后的信号形状
print("滑窗后信号形状：", windows.shape)
#print(windows)

滑窗后信号形状： (32, 3000, 126)


In [10]:
import onnx
import onnxruntime
save_path = './model_saved/cnn_3000_100e_1.pt'

onnx_file_name = "./cnn_3000_100e.onnx"
torch.load(save_path, map_location=torch.device('cpu'))

OrderedDict([('conv1.weight',
              tensor([[[[ 0.4752,  0.3993, -0.2588,  0.1631]]],
              
              
                      [[[-0.4077,  0.4999, -0.1816, -0.0157]]],
              
              
                      [[[-0.1810, -0.4854,  0.3795,  0.3577]]],
              
              
                      [[[ 0.0896, -0.1243,  0.3935,  0.0694]]],
              
              
                      [[[ 0.1553, -0.2519,  0.2261,  0.3780]]],
              
              
                      [[[ 0.3069,  0.2198,  0.2584, -0.4192]]],
              
              
                      [[[-0.4673,  0.3854,  0.2515, -0.4788]]],
              
              
                      [[[ 0.3410, -0.4043,  0.4628,  0.0794]]],
              
              
                      [[[ 0.0493, -0.0526, -0.4163,  0.4766]]],
              
              
                      [[[-0.3292,  0.4598,  0.1505,  0.4126]]],
              
              
                      [[[ 0.29

In [11]:
import onnx
# 我们可以使用异常处理的方法进行检验
try:
    # 当我们的模型不可用时，将会报出异常
    onnx.checker.check_model(onnx_file_name)
except onnx.checker.ValidationError as e:
    print("The model is invalid: %s"%e)
else:
    # 模型可用时，将不会报出异常，并会输出“The model is valid!”
    print("The model is valid!")

The model is valid!


In [12]:
for i in range(windows.shape[2]):
    signal = pd.DataFrame(windows[:,:,i])
    model = EEGNet()
    model.load_state_dict(torch.load(save_path))
    model.eval()
    input = torch.from_numpy(signal.values)
    input = input.unsqueeze(0)
    with torch.no_grad():
        output = model(input.float())
        #print(output)
    # probabilities = torch.nn.functional.softmax(output, dim=1)
    # print(probabilities)

    _, pred = torch.max(output, dim=1)  # 找到预测分数最大的类别，得到预测类别
    label_map = {0: 'lefthand', 1:'read' ,  2:'rest', 3: 'walkbase', 4: 'walkl' ,5: 'walkfocus'}
    print(label_map[pred.item()])

walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkl
walkbase
walkbase
walkbase
walkbase
walkl
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
rest
walkbase
walkbase
walkl
walkbase
walkl
walkl
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
read
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
lefthand
lefthand
read
walkbase
walkfocus
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
walkbase
